# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [2]:
# Imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import os
import sys
import json
from collections import Counter
import random

## Scikit Learn Imports.........
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.utils import class_weight

## Keras Imports...........
import keras
import keras.backend as K
from keras.layers import Dense,Flatten,Dropout,BatchNormalization,Input,Lambda,Merge,Conv1D
from keras.models import Sequential,Model
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam,SGD,RMSprop
from collections import Counter

%matplotlib inline
plt.rcParams['figure.figsize'] = 15,8

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

/home/sunny/anaconda2/envs/p3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [8]:
## Readinf flower names and file names
data_dir = os.path.join(os.getcwd(),'jpg')
train_dir = os.path.join(data_dir,'train')
valid_dir = os.path.join(data_dir,'valid')
test_dir = os.path.join(data_dir,'test')

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
#print(cat_to_name)
labels = loadmat('imagelabels.mat')
labels = labels['labels'][0]
print('Total images : %d' % len(labels))

files = list(os.walk(data_dir))[0][2]
#print(files)
files = sorted(files)

Total images : 8189


In [3]:
#%ls
## Renaming files with proper flower names.....

for file in files:
    file_idx = int(file.split('_')[1].split('.')[0])
    try:
        flower_name = cat_to_name[str(labels[file_idx])].replace(' ','_')
        file_new = file.replace('image',flower_name)
        #print(file)
        os.rename(os.path.join(data_dir,file),os.path.join(data_dir,file_new))
    except IndexError:
        os.remove(os.path.join(data_dir,file))
        print('No Image name found for image index : %d. Deleting File....' % file_idx)
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
#print(type(labels))
#print(labels)

In [4]:
## Moving flowers to thrie folders.....
files = list(os.walk(data_dir))[0][2]

if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
for flower_name in cat_to_name.values():
    new_flower_name = flower_name.replace(' ','_')
    same_flowers = [file for file in files if new_flower_name == file[:-10]]
    new_dir = os.path.join(train_dir,new_flower_name)
    #print('Creating directory : %s' % new_dir)
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
    if same_flowers:
        for flower in same_flowers:
            os.rename(os.path.join(data_dir,flower),os.path.join(new_dir,flower))

FileExistsError: [Errno 17] File exists: '/home/sunny/Desktop/Python/jupyter_notebooks/freelancing_projects/Pytorch deeplearning guidance/aipnd-project/jpg/train'

In [ ]:
#print(data_dir)
if not os.path.exists(valid_dir):
    os.mkdir(valid_dir)
    
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
    
folder_list = list(os.walk(train_dir))[0][1]

## Moving 10% files from train folder to valid folder by shuffling images
for folder in folder_list:
    full_path = os.path.join(train_dir,folder)
    valid_path = full_path.replace('train','valid')
    if not os.path.exists(valid_path):
        os.mkdir(valid_path)
    files = os.listdir(full_path)
    random.shuffle(files)
    for i in range(int(len(files)*.1)):
        os.rename(os.path.join(full_path,files[i]), os.path.join(valid_path,files[i]))

## Moving 10% files from train folder to test folder by shuffling images
for folder in folder_list:
    full_path = os.path.join(train_dir,folder)
    test_path = full_path.replace('train','test')
    if not os.path.exists(test_path):
        os.mkdir(test_path)
    files = os.listdir(full_path)
    random.shuffle(files)
    for i in range(int(len(files)*.1)):
        os.rename(os.path.join(full_path,files[i]), os.path.join(test_path,files[i]))

In [9]:
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomVerticalFlip(),
                                     #transforms.RandomRotation(), ## Need to find out rotation param
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
valid_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

#transforms.CenterCrop((224,224))
train_dataset = torchvision.datasets.ImageFolder(train_dir,transform=train_transforms)
valid_dataset = torchvision.datasets.ImageFolder(valid_dir,transform=valid_transforms)
test_dataset  = torchvision.datasets.ImageFolder(test_dir,transform=test_transforms)

#loader = train_dataset.loader
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=16,
                                          shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset) #,
                                          #batch_size=16,
                                          #shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset) #,
                                          #batch_size=16,
                                          #shuffle=True)

In [33]:
train_dataset.class_to_idx == valid_dataset.class_to_idx, train_dataset.class_to_idx == test_dataset.class_to_idx, test_dataset.class_to_idx == valid_dataset.class_to_idx

(True, True, True)

In [34]:
train_dataset.class_to_idx

{'alpine_sea_holly': 0,
 'anthurium': 1,
 'artichoke': 2,
 'azalea': 3,
 'ball_moss': 4,
 'balloon_flower': 5,
 'barbeton_daisy': 6,
 'bearded_iris': 7,
 'bee_balm': 8,
 'bird_of_paradise': 9,
 'bishop_of_llandaff': 10,
 'black-eyed_susan': 11,
 'blackberry_lily': 12,
 'blanket_flower': 13,
 'bolero_deep_blue': 14,
 'bougainvillea': 15,
 'bromelia': 16,
 'buttercup': 17,
 'californian_poppy': 18,
 'camellia': 19,
 'canna_lily': 20,
 'canterbury_bells': 21,
 'cape_flower': 22,
 'carnation': 23,
 'cautleya_spicata': 24,
 'clematis': 25,
 "colt's_foot": 26,
 'columbine': 27,
 'common_dandelion': 28,
 'corn_poppy': 29,
 'cyclamen': 30,
 'daffodil': 31,
 'desert-rose': 32,
 'english_marigold': 33,
 'fire_lily': 34,
 'foxglove': 35,
 'frangipani': 36,
 'fritillary': 37,
 'garden_phlox': 38,
 'gaura': 39,
 'gazania': 40,
 'geranium': 41,
 'giant_white_arum_lily': 42,
 'globe-flower': 43,
 'globe_thistle': 44,
 'grape_hyacinth': 45,
 'great_masterwort': 46,
 'hard-leaved_pocket_orchid': 47,
 '

In [10]:
# Testing loaders
for images,labels in train_loader:
    print(images.shape,labels.shape)
    break
for images,labels in valid_loader:
    print(images.shape,labels.shape)
    break
for images,labels in test_loader:
    print(images.shape,labels.shape)
    break

torch.Size([16, 3, 224, 224]) torch.Size([16])
torch.Size([1, 3, 224, 224]) torch.Size([1])
torch.Size([1, 3, 224, 224]) torch.Size([1])


In [12]:
# TODO: Define your transforms for the training, validation, and testing sets
#data_transforms = 

# TODO: Load the datasets with ImageFolder
#image_datasets = 

# TODO: Using the image datasets and the trainforms, define the dataloaders
#dataloaders =
len(train_loader),len(train_loader.dataset),len(test_loader.dataset), len(valid_loader.dataset)

(381, 6092, 630, 1466)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [13]:
#import json

#with open('cat_to_name.json', 'r') as f:
#    cat_to_name = json.load(f)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [14]:
vgg16_with_batch_norm = torchvision.models.vgg16_bn(pretrained=True).to(device) ## Keep pretrained to True to get Vgg16 imagenet weights

In [20]:
for param in vgg16_with_batch_norm.parameters():
    param.requires_grad_(False)

In [21]:
print(vgg16_with_batch_norm.classifier)
vgg16_with_batch_norm.classifier.add_module('7',nn.Linear(in_features=1000,out_features=102,bias=True))
vgg16_with_batch_norm.classifier.add_module('8',nn.Softmax(dim=1))
print(vgg16_with_batch_norm.classifier)
for param in vgg16_with_batch_norm.classifier.parameters():
    param.requires_grad_(True)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
  (7): Linear(in_features=1000, out_features=102, bias=True)
  (8): Softmax()
)


In [22]:
# TODO: Build and train your network
entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(params=vgg16_with_batch_norm.classifier.parameters(),lr=0.001)

In [ ]:
loss = 0
epochs = 10
for i in range(epochs):
    running_loss = 0.0
    correct = 0
    for images,labels in train_loader:
        #images,labels = Variable(images),Variable(labels)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16_with_batch_norm(images)
        loss =entropy_loss(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        #print(outputs.size())
        #print(outputs)
        #print(labels.size())
        #print(labels)
        outputs = [np.argmax(i) for i in outputs.data]
        correct += (labels.numpy() == outputs).sum()
        print('Loss : %s | Correct : %s' % (running_loss, correct))
    final_loss = running_loss / len(train_loader)
    print('Training Loss : %s  |  Training accuracy : %s' % (final_loss, correct / len(train_loader.dataset)))
        
    with torch.no_grad():
        valid_loss = 0.0
        for images.labels in valid_loader:
            #images,labels = Variable(images),Variable(labels)
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16_with_batch_norm(images)
            loss = entropy_loss(outputs,labels)
            valid_loss += loss.data[0]
            correct += (labels == outputs).sum()
        final_loss = valid_loss / len(valid_loader.dataset)
        print('Validation Loss : %s  |  Validation accuracy : %s' % (final_loss, correct / len(valid_loader.dataset)))

/home/sunny/anaconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


Loss : tensor(4.6417) | Correct : 0
Loss : tensor(9.1584) | Correct : 2
Loss : tensor(13.7375) | Correct : 3
Loss : tensor(18.3167) | Correct : 4
Loss : tensor(22.9584) | Correct : 4
Loss : tensor(27.6001) | Correct : 4


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
with torch.no_grad():
    test_loss = 0.0
    for images.labels in test_loader:
        #images,labels = Variable(images),Variable(labels)
        images, labels = images.to(device), labels.to(device)
        outputs = vgg16_with_batch_norm(images)
        loss = entropy_loss(outputs,labels)
        test_loss += loss.data[0]
        correct += (labels == outputs).sum()
    final_loss = test_loss / len(test_loader.dataset)
    print('Validation Loss : %s  |  Validation accuracy : %s' % (final_loss, correct / len(test_loader.dataset)))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [25]:
# TODO: Save the checkpoint 

model_state_dict = vgg16_with_batch_norm.state_dict()
optimizer_state_dict = optimizer.state_dict()
loss_state_dict = entropy_loss.state_dict()

checkpoint_dict = {
    'epochs': epochs,
    'model_dict': model_state_dict,
    'optimizer' : optimizer_state_dict,
    'loss' : loss_state_dict,
}
torch.save(checkpoint_dict,'first_run.pth.tar')

In [35]:
model_state_dict.keys(), optimizer_state_dict.keys(), loss_state_dict.keys()

(odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.3.weight', 'features.3.bias', 'features.4.weight', 'features.4.bias', 'features.4.running_mean', 'features.4.running_var', 'features.7.weight', 'features.7.bias', 'features.8.weight', 'features.8.bias', 'features.8.running_mean', 'features.8.running_var', 'features.10.weight', 'features.10.bias', 'features.11.weight', 'features.11.bias', 'features.11.running_mean', 'features.11.running_var', 'features.14.weight', 'features.14.bias', 'features.15.weight', 'features.15.bias', 'features.15.running_mean', 'features.15.running_var', 'features.17.weight', 'features.17.bias', 'features.18.weight', 'features.18.bias', 'features.18.running_mean', 'features.18.running_var', 'features.20.weight', 'features.20.bias', 'features.21.weight', 'features.21.bias', 'features.21.running_mean', 'features.21.running_var', 'features.24.weight', 'features.

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
## When you load the model again than remember to do model.eval() otherwise results will differ

checkpoint = torch.load('first_run.pth.tar')
model.load_state_dict(checkpoint['model_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
loss.load_state_dict(checkpoint['loss'])

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes